# EE123 Lab 2: Software Defined Radio - Part II

### Written by Miki Lustig and Frank Ong
Updated Drake Lin, Spring 2023

Updated by Josh Sanz, Spring 2020, Spring 2022

Updated by Alan Dong, Spring 2021

### About the lab:

In the first part of the lab, we will look at the power spectrum of the NOAA radio signal and introduce settings of the SDR along the way. In the second part of the lab, we will decode Mode-S ADS-B packets that are transmitted by airplanes passing by and picked up by the SDR, which allows us to track airplanes in real time.


## About the SDR

The rtl-sdr usb dongle enables you to obtain samples from the electromagnetic spectrum around you. In very general terms, the dongle contains several components:

1. The antenna couples to received electromagnetic fields and tiny currents are produced in it. 
2. A tuner integrated circuit amplifies the signal, filters it, demodulates it to an intermediate frequency where it is filtered again. The dongles we distributed in class contain either the Rafael Micro 820T (Black dongles) tuner or the Elonics E4000  (white dongles). 
3. All dongles are equipped with the realtek RTL2832U (hence rtl-sdr). Although the chip is capable of doing many things (like decoding TV in Europe), we use only its analog to digital converters and its USB interface. The RTL2832U samples the signal that is coming from the tuner and spits out samples to the computer through the USB interface. 

The SDR returns samples at a desired rate up to 2.4MS/s (sometimes 3.5MS/s) of a part of the spectrum around a desired center frequency. For example, setting a center frequency $f_0 = 88.5\cdot 10^6$ and a sampling rate of $Fs=2\cdot 10^6$ will result in a complex valued sequence $x[n]$ whose DTFT corresponds to the physical frequency range of $87.5\cdot 10^6 < f < 89.5\cdot 10^6$. In other words, the digital frequency $\omega=0$ of $X(e^{j\omega})$, the DTFT of $x[n]$, will correspond to the physical frequency $88.5$MHz. The digital frequency $\omega=\pi$ will correspond to $89.5$MHz and $\omega=-\pi$ will correspond to $87.5$MHz.

Q) Why is the sequence $x[n]$ complex valued ??? 

A) Consider the case when there is a transmitter which outputs a pure frequency at 89MHz. We choose a center frequency of 88.5MHz and sampling rate of 2MHz. The spectrum of $x[n]$ will not be symmetric, so $x[n]$ has to be complex valued! The received signal would be $x[n] = e^{j(2\pi500000/2000000)n} = e^{j(\pi/2)n}$ which will have a single frequency at $\omega=\pi/2$ --> corresponding to 89MHz. 



To learn about what you can do with SDR's, I  recommend you watch this youtube video. Most (not all) the stuff shown there can be done using rtl-sdr. At minute 5:00 you will see an example of ADS-B, which you will partly implement in this lab. (You will need internet connection to watch the video)

In [1]:
from IPython.display import YouTubeVideo
# A video on what you can do with Software defined radio. The B200 is a high-end SDR which is capable to much more than the rtl-sdr. 
# however, most of the stuff shown in the video could be done with the rtl-sdr as well. 
YouTubeVideo('cygDXeZaiOM')

In [1]:
# Import functions and libraries
import numpy as np, matplotlib.pyplot as plt
from numpy import *
from numpy.fft import *
from matplotlib.pyplot import *
from rtlsdr import RtlSdr

%matplotlib inline

## Task I: Capture data and compute the spectrum
Let's look at a simple example of acquring data and looking at its spectrum. NOAA weather radio is transmitted from San Francisco at 162.4MHz. It is a 5KHz frequency modulated (FM) audio signal that voices a recording of the weather. If you live in the East Bay, you might also be able to pick up the Mt. Diablo transmitter at 162.425MHz. Here's a map of the SF coverage:

<img src="https://inst.eecs.berkeley.edu/~ee123/sp14/lab/KHB49.png" alt="Drawing" style="height: 500px;"/>

It's best if you are outside of the building when collecting samples. Let's collect 4 seconds worth of samples, sampled with a center frequency of 162MHz and a sampling rate of 960KHz.

**Plug in the SDR and attach the antenna.** Then, we instantiate the sdr with sampling rate and center frequency parameters by creating the rtlsdr object as below:
```
sdr = RtlSdr()
sdr.set_sample_rate(960000)    # sampling rate
sdr.set_center_freq(162200000)   # 162.2MhZ center frequency
```
* **Create an RtlSdr object called sdr** with the given parameters.

In [2]:
fs = 960000.
fc = 162200000.

# Your code here:



It is also useful to set the gain appropriately. The default is usually quite high and the signal saturates. 

To see what the available gains are, invoke: 

```
print(sdr.valid_gains_db)
```

The result will depend on the type of SDR you have (there are a few variants)

In [3]:
print(sdr.valid_gains_db)

Pick a number that's one or two slots larger than the middle (around 37.2 may work well) and 
* **Set the gain** using `sdr.set_gain(gain)`
    

In [4]:
# Set the gain
gain = TODO
sdr.set_gain(gain)

The python interface to the SDR requires that the number of samples be a multiple of 256. To read samples from the SDR, you can use the command `sdr.read_samples(N_samples)`.

* **Read 4 seconds of data** from the SDR. Remember that we are sampling at 960000Hz.

If you are done with the device, you can clear it by:
    sdr.close()

In [5]:
# Record 4 seconds of data
N_samples = TODO
y = TODO

sdr.close()

# You may want to save your capture to work with later.
np.save('noaa_mine.npy',y)

If you're working in the lab, the recording probably won't have good data, so run the line below to load Miki's capture and continue. 
If you have the chance to, feel free to record some data at home!

In [19]:
# y = np.load('../data/noaa.npy')

Here is code to load your own capture:

In [ ]:
y = np.load('noaa_mine.npy')

We will only use a small portion of our recording to process and look at the spectrum. Let’s crop the recording and analyze the spectrum of the first 16384 samples. We can use the function `fft` to compute the DFT and then `fftshift` to center the DFT around $\omega=0$. Remember, the signal you captured is complex-valued, and hence has a two-sided spectrum. 

* **Crop the recording to the first 16384 samples.**
* **Apply a Kaiser$(\beta=6)$ window then compute the DFT, centered around $\omega=0$.**
* **Plot in log scale with physical frequencies $f$**, not $\omega$ on the x-axis.

Instead of showing the spectrum in terms of $\omega$, we plot in terms of the physical frequencies to make identifying signal sources and adjusting the SDR easier. The function `plt.semilogy` plots only the y-axis in log scale.

In [15]:
# Your code here:

# End of your code

# Plot
width, height = figaspect(0.2)
fig=plt.figure(figsize=(width,height))
p = plt.semilogy( f/1e6, abs(Y_w))
plt.xlabel('frequency [MHz]')
plt.axis('tight')
plt.title('Spectrum');

You should be able to see some energy in the spectrum around the right frequency. However, what you should notice immediately is that the spectrum is noisy. In addition, there might be some spurious peaks, and in the white dongles (E4000) there is also a large peak at the DC frequency. The peak at the DC frequency is due to constant bias in the ADC of the device. This translates to an impulse at $\omega = 0$. Spurious peaks come from local oscillator leakage and also quantization errors. 

To reduce the noise and get a finer look at the spectrum, we will break our **entire sequence (not the cropped one)** into smaller sections and average. We don't need a very good spectral resolution, so we will break it into chunks of 2048 samples, calculate their magnitude spectra, and average. This is also called the average power specrum.

* **Reshape the sequence into a matrix with row size of 2048.** Remember that the ordering of the matrix is 'C' style and hence row-first. 
* **Multiply the rows by a kaiser window with $\beta=6$.**
* **Perform `fft` and `fftshift` along the 2nd dimension.**
* Compute the absolute square of the result and average along the 1st dimension
* **Plot the result with physical frequencies**

In [16]:
# Calculate the DFT of each 2048 sample section of the recording.
# Your code here:

# End of your code

# Compute mean
Pspect = mean(abs(X)*abs(X),axis=0)

In [17]:
# Display:
f = TODO

width, height = figaspect(0.3)
plt.figure(figsize=(width,height))
plt.semilogy(f/1e6, Pspect)
plt.xlabel('frequency [MHz]') 
plt.title('Average Power Spectrum');

This plot is called the average power spectrum. 
As you can see, the spectrum looks much nicer now. 
Can you identify the NOAA weather station? 
Only the wide spectrum peaks are real. 
The very narrow spiky peaks in the spectrum are probably due to leakage of the local oscilator, non-linearity in the receiver, and poor dynamic range of the ADC of the device. 
Still, for $12 it is not too bad!

What is the spectral resolution in the plot compared to the original plot?



* **Repeat the the measurement and the average power spectrum for the center frequency 88.4MHz.** Can you identify NPR's 88.5MHz station?

Note: The rectangular bands you see around the main signal are HD radio signals. These are transmitted at the band edges and provide digital radio programming. Unfortunately HD radio in the USA uses proprietery protocol, and we can not easily decode it at this time. If you want to implement this yourself, you can visit [this](https://www.rtl-sdr.com/decoding-and-listening-to-hd-radio-nrsc-5-with-an-rtl-sdr/) link and follow the instructions there. 

In [22]:
# Your code here:



Again, feel free to use Miki's capture if your setup isn't working

In [21]:
# y = np.load('../data/npr.npy')

Here is code to load your own capture:

In [ ]:
y = np.load('npr_mine.npy')

In [22]:
# Split, window, and FFT your recording
# Your code here:

# End of your code

# Compute mean
Pspect = TODO
f = TODO

width, height = figaspect(0.2)
fig=plt.figure(figsize=(width,height))
p = plt.semilogy(f/1e6,Pspect)
plt.xlabel('frequency [MHz]'); plt.title('Average Power Spectrum')


## You are now ready to proceed to the next part. Make sure you kill this kernel to save memory

In [23]:
%reset -f